In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Below codes created by me

In [2]:
import tensorflow as tf
print(tf.__version__)

2.4.0


In [3]:
import numpy as np

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
data, info = tfds.load(name='imdb_reviews', as_supervised=True, with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAEYSMK/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAEYSMK/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAEYSMK/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [13]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [14]:
data

{'test': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>}

In [5]:
# split data
train, test = data['train'], data['test']

In [6]:
train_sentences, train_labels = [], []
test_sentences, test_labels = [], []

In [7]:
# for loop to extract each sentence and label
# use .numpy() to convert tensor to numpy
# use .decode('utf8') to convert byte to string
for sent, label in train:
    train_sentences.append(sent.numpy().decode('utf8')) 
    train_labels.append(label.numpy())

for sent, label in test:
    test_sentences.append(sent.numpy().decode('utf8'))
    test_labels.append(label.numpy())

In [8]:
train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)

In [9]:
# set hyperparameters for tokenizing
vocab_size = 20000
oov_token = '<OOV>'
max_length = 30
embedding_dim = 16
trunc_type = 'post'
padding = 'post'

In [10]:
# initiate and fit Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

# get word_index
word_index = tokenizer.word_index

# tokenize on train_sentences and pad them
sequences = tokenizer.texts_to_sequences(train_sentences)
sequences_padded = pad_sequences(sequences, maxlen=max_length, padding=padding, truncating=trunc_type)

# tokenize on test_sentences and pad them
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding, truncating=trunc_type)

In [11]:
# define the inverse of word_index and the function to decode encoded text
reverse_word_index = {value:key for key, value in word_index.items()}

def decode_review(encoded_text):
    return ' '.join([reverse_word_index[i] for i in encoded_text])

In [12]:
# let's see an example
print(decode_review(sequences_padded[10]))
print(train_sentences[10])

during a sleepless night i was switching through the channels found this embarrassment of a movie what were they thinking br br if this is life after remote control for
During a sleepless night, I was switching through the channels & found this embarrassment of a movie. What were they thinking?<br /><br />If this is life after "Remote Control" for Kari (Wuhrer) Salin, no wonder she's gone nowhere.<br /><br />And why did David Keith take this role? It's pathetic!<br /><br />Anyway, I turned on the movie near the end, so I didn't get much of the plot. But this must've been the best part. This nerdy college kid brings home this dominatrix-ish girl...this scene is straight out of the comic books -- or the cheap porn movies. She calls the mother anal retentive and kisses the father "Oh, I didn't expect tongue!" Great lines!<br /><br />After this, I had to see how it ended..<br /><br />Well, of course, this bitch from hell has a helluva past, so the SWAT team is upstairs. And yes...they surr

In [42]:
# define a model
model = tf.keras.Sequential(name='Course3_Week2_Lesson1_model')
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(tf.keras.layers.GlobalAveragePooling1D())
# model.add(tf.keras.layers.Dense(units=256, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
# model.add(tf.keras.layers.Dropout(rate=0.5))
# model.add(tf.keras.layers.Dense(units=16, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [43]:
model.summary()

Model: "Course3_Week2_Lesson1_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 30, 16)            320000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 17        
Total params: 320,017
Trainable params: 320,017
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [30]:
# define callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') > 0.98:
            print('\nAccuracy reached 98%. Training terminated')
            self.model.stop_training = True

callback = myCallback()

In [45]:
model.fit(x=sequences_padded, 
          y=train_labels_final, 
          batch_size=32, 
          epochs=10, 
        #   callbacks=[callback], 
          validation_data=(test_sequences_padded, test_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 8ms/step - loss: 0.6747 - acc: 0.6157 - val_loss: 0.6000 - val_acc: 0.6944
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.5498 - acc: 0.7514 - val_loss: 0.5429 - val_acc: 0.7257
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4641 - acc: 0.7972 - val_loss: 0.5270 - val_acc: 0.7316
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4072 - acc: 0.8298 - val_loss: 0.5256 - val_acc: 0.7353
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3730 - acc: 0.8427 - val_loss: 0.5350 - val_acc: 0.7326
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3360 - acc: 0.8647 - val_loss: 0.5483 - val_acc: 0.7298
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3152 - acc: 0.8723 - val_loss: 0.5672 - val_acc: 0.7259
Epoch 8/10
782/782 [==============================] - 5s 7ms/step - loss: 0.

In [50]:
# get embedding layer's weights and its shape
emb_layer = model.layers[0].get_weights()[0]
emb_layer.shape

(20000, 16)

In [57]:
test_pred = test_sequences_padded[-3:]

preds = model.predict(x=test_pred)

for r, t, p, l in zip(test_sentences[-3:], test_pred, preds, test_labels[-3:]):
    print('real text:', r)
    print('padded sequence:', t)
    print('encoded - padded seqence:', decode_review(t))
    print('prediction:', p)
    print('pred label: negative review' if p < 0.5 else 'pred label: positive review')
    print('true label:', l)
    print('true label: negative review' if l == 0 else 'true label: positive review')
    print('\n')

real text: Most predicable movie I've ever seen...extremely boring, I feel like I've seen a hundred movies with the same storyline as this one. Acting is OK at best, there's no action really and there is definitely no thrills. Capable actors with terrible script i think it could have been written better by a 10th grader. Felt like more of a chore to watch because I was hoping that there would be something in this movie that was going to set it apart from all the other garbage but this fit right in on the heap. The whole movie I was waiting for something good to happen but it never came. I never rate movies and I never review movies but this movie was so bad that i had to log in here and post a review to try and save a few poor souls from wasting their time (and/or money) with this movie. I pirated it and wish I never even wasted the hard drive space. If I spent 10 bucks to see this in theaters I would kill myself.
padded sequence: [  89    1   18  205  124  108  574  355   11  233   38

# Below codes created by Laurence

In [ ]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


'this'

In [46]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(sequences_padded[3])
print(decode_review(sequences_padded[3]))
print(' '.join([reverse_word_index[i] for i in sequences_padded[3]]))
print(train_sentences[3])

[   12     7     2   241     5    20    16     4  8776  2707  2653    52
     2   358     5     2   180    68   138  1401    17    92   203   968
    15    23 18512    81     4   192]
this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big
this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [32]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model1.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 16)            320000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 480)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 2886      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 7         
Total params: 322,893
Trainable params: 322,893
Non-trainable params: 0
_________________________________________________________________


In [34]:
num_epochs = 10
model1.fit(sequences_padded, train_labels_final, epochs=num_epochs, validation_data=(test_sequences_padded, test_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.6693 - accuracy: 0.5644 - val_loss: 0.5339 - val_accuracy: 0.7236
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3883 - accuracy: 0.8334 - val_loss: 0.5746 - val_accuracy: 0.7126
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1955 - accuracy: 0.9400 - val_loss: 0.7064 - val_accuracy: 0.7000
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0624 - accuracy: 0.9889 - val_loss: 0.8944 - val_accuracy: 0.6866
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0181 - accuracy: 0.9987 - val_loss: 1.0478 - val_accuracy: 0.6862
Epoch 6/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0054 - accuracy: 0.9998 - val_loss: 1.1484 - val_accuracy: 0.6854
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 1.2521 - val_accuracy: 0.6879
Epoch 

In [59]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(20000, 16)


In [60]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [62]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
